In [ ]:
# SVD

import numpy as np
import cv2
from matplotlib import pyplot as plt
from google.colab import drive

def cartoonify_with_svd(image_path, k=50):
    # Read the image from the drive through google colab
    img = cv2.imread(image_path)  # Read the image as a color image

    # Convert the image to grayscale
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply SVD to Perform Singular Value Decomposition (SVD) on the image. U, S, and Vt represent the left singular vectors, singular values, and right singular vectors, respectively
    U, S, Vt = np.linalg.svd(img_gray, full_matrices=False)

    # Retain only the first k singular values
    S[:k] = S[:k] / S[0]
    S[k:] = 0

    # Reconstruct the image
    img_svd = np.dot(U, np.dot(np.diag(S), Vt))

    # Normalize the image values to the range [0, 255] and convert the data type to unsigned 8-bit integers.
    img_svd = cv2.normalize(img_svd, None, 0, 255, cv2.NORM_MINMAX)
    img_svd = img_svd.astype(np.uint8)

    # Apply a bilateral filter to the cartoonified image for smoothing.
    img_cartoon = cv2.bilateralFilter(img_svd, d=9, sigmaColor=75, sigmaSpace=75)

    # Display the original, SVD-based cartoon, and smoothed cartoon images
    plt.figure(figsize=(15, 5))

    plt.subplot(1, 3, 1)
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.title('Original Image')

    plt.subplot(1, 3, 2)
    plt.imshow(img_svd, cmap='gray')
    plt.title('Cartoon (SVD)')

    plt.subplot(1, 3, 3)
    plt.imshow(img_cartoon, cmap='gray')
    plt.title('Cartoon (Smoothed)')

    plt.show()

drive.mount('/content/drive')
# Specify the correct image path with PNG extension
image_path = '/content/drive/MyDrive/p.png'
cartoonify_with_svd(image_path, k=50)


In [ ]:


#k-means
import numpy as np
import cv2
from matplotlib import pyplot as plt
from google.colab import drive

drive.mount('/content/drive')

def read_file(filename):
    img = cv2.imread(filename)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img)
    plt.show()
    return img

filename = '/content/drive/MyDrive/p.png'
img = read_file(filename)

def edge_mask(img, line_size, blur):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    gray_blur = cv2.medianBlur(gray, blur)
    edges = cv2.adaptiveThreshold(gray_blur, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, line_size, blur)
    return edges

line_size, blur_value = 7, 7
edges = edge_mask(img, line_size, blur_value)
plt.imshow(edges, cmap="gray")
plt.show()

total_colors = 30
line_wdt = 9
blur_value = 7

def color_quantisation(img, k):
    data = np.float32(img).reshape((-1, 3))
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 20, 0.001)
    _, labels, centers = cv2.kmeans(data, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
    centers = np.uint8(centers)
    segmented_image = centers[labels.flatten()]
    quantized_image = segmented_image.reshape(img.shape)
    return quantized_image

img = color_quantisation(img, total_colors)
plt.imshow(img)
plt.show()

blurred = cv2.bilateralFilter(img, d=line_wdt, sigmaColor=200, sigmaSpace=200)
plt.imshow(blurred)
plt.show()

def cartoon():
    c = cv2.bitwise_and(blurred, blurred, mask=edges)
    plt.imshow(c)
    plt.title("Reanimated_Image")
    plt.show()
    plt.imshow(img)
    plt.title("Original_Image")
    plt.show()

cartoon()